## 1. Import Required Libraries

In [1]:
import os
import sys
import subprocess
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, Image

# Set visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")
print(f"Current working directory: {os.getcwd()}")

✓ All libraries imported successfully
Current working directory: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/nordbyen_heat_benchmark/notebooks


## 2. Configure Benchmark Parameters

In [2]:
# Data and results paths (relative to project root)
DATA_PATH = "../../processing/nordbyen_processing/nordbyen_features_engineered.csv"
RESULTS_DIR = "../results"
SCRIPTS_DIR = "../scripts"

# ========================================
# MODELS TO BENCHMARK - Choose strategy:
# ========================================
# Option 1: All models in one job (may take 15-20 hours)
# MODELS_TO_RUN = ["NHITS_Q", "NHITS_MSE", "TIMESNET_Q", "TIMESNET_MSE"]

# # Option 2: Split by architecture (RECOMMENDED - run both jobs in parallel)
# # Job 1: NHITS models (~5-10 hours)
# MODELS_TO_RUN = ["NHITS_Q", "NHITS_MSE"]

# Job 2: TimesNet models (~5-10 hours) - Change to this and run cell 3 again
MODELS_TO_RUN = ["TIMESNET_Q", "NHITS_Q"]

# SLURM job configuration
SLURM_SCRIPT = "../scripts/benchmark_job.slurm"
JOB_ID_FILE = "../scripts/current_job_id.txt"

# Verify data file exists
if os.path.exists(DATA_PATH):
    print(f"✓ Data file found: {DATA_PATH}")
    df_info = pd.read_csv(DATA_PATH, nrows=5)
    print(f"  Columns: {list(df_info.columns[:5])}... (total: {len(df_info.columns)})")
    print(f"  Target: heat_consumption")
else:
    print(f"✗ Data file not found: {DATA_PATH}")
    
# Create results directory if needed
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"✓ Results directory ready: {RESULTS_DIR}")
print(f"✓ Benchmarking models: {', '.join(MODELS_TO_RUN)}")

✓ Data file found: ../../processing/nordbyen_processing/nordbyen_features_engineered.csv
  Columns: ['timestamp', 'heat_consumption', 'temp', 'dew_point', 'humidity']... (total: 27)
  Target: heat_consumption
✓ Results directory ready: ../results
✓ Benchmarking models: TIMESNET_Q, NHITS_Q


## 3. Submit SLURM Benchmark Job

**RECOMMENDED**: Submit to SLURM for full benchmarking on GPU

In [9]:
# Submit benchmarker to SLURM
print("Submitting heat benchmarker job to SLURM...")
print(f"Models to run: {', '.join(MODELS_TO_RUN)}")
print(f"Script: {SLURM_SCRIPT}")

# Convert models list to space-separated string for shell argument
models_arg = " ".join(MODELS_TO_RUN)

result = subprocess.run(
    ["sbatch", SLURM_SCRIPT, models_arg],
    capture_output=True,
    text=True,
    cwd="../scripts"
)

if result.returncode == 0:
    # Extract job ID from output: "Submitted batch job 123456"
    job_id = result.stdout.strip().split()[-1]
    print(f"\n✓ Job submitted successfully!")
    print(f"  Job ID: {job_id}")
    print(f"  Models: {', '.join(MODELS_TO_RUN)}")
    print(f"  Log files: {SCRIPTS_DIR}/benchmark_{job_id}.log / benchmark_{job_id}.err")
    print(f"\n💡 Next: Run cell below to monitor job status")
    
    # Save job ID for monitoring
    with open(JOB_ID_FILE, "w") as f:
        f.write(job_id)
else:
    print(f"✗ Error submitting job:\n{result.stderr}")

Submitting heat benchmarker job to SLURM...
Models to run: TIMESNET_Q, NHITS_Q
Script: ../scripts/benchmark_job.slurm

✓ Job submitted successfully!
  Job ID: 1477203
  Models: TIMESNET_Q, NHITS_Q
  Log files: ../scripts/benchmark_1477203.log / benchmark_1477203.err

💡 Next: Run cell below to monitor job status


## 4. Monitor Job Status

Check if the benchmarker job is running or complete.

In [3]:
# Check SLURM job status
if os.path.exists(JOB_ID_FILE):
    with open(JOB_ID_FILE, "r") as f:
        job_id = f.read().strip()
else:
    print("❌ No job ID found. Run cell above to submit the job.")
    job_id = None

if job_id:
    print(f"Checking status of Job {job_id}...")
    print("="*70)
    
    # Check with squeue
    result = subprocess.run(
        ["squeue", "-j", job_id, "--format=%.18i %.9P %.20j %.8u %.8T %.10M %.9l %.6D %R"],
        capture_output=True,
        text=True
    )
    
    if "Invalid job id" in result.stderr or not result.stdout.strip().split('\n')[1:]:
        print(f"⏹️  Job {job_id} is no longer in queue (completed or failed)")
        
        # Check if results exist
        results_file = os.path.join(RESULTS_DIR, "benchmark_results.csv")
        if os.path.exists(results_file):
            print(f"\n✅ Job completed successfully!")
            print(f"   Results file found: {results_file}")
            print(f"\n💡 Next: Continue to next cell to view results")
        else:
            print(f"\n❌ Job completed but results not found!")
            print(f"   Check log files: {SCRIPTS_DIR}/benchmark_{job_id}.log")
    else:
        print(result.stdout)
        print(f"\n⏳ Job is still running. Re-run this cell to check again.")
        print(f"   Estimated time: ~2-3 hours for all 4 models")
        
    # Show recent log output
    log_file = f"{SCRIPTS_DIR}/benchmark_{job_id}.log"
    if os.path.exists(log_file):
        print(f"\n📄 Recent log output (last 15 lines):")
        print("-"*70)
        result = subprocess.run(["tail", "-n", "15", log_file], capture_output=True, text=True)
        print(result.stdout)

Checking status of Job 1477203...
             JOBID PARTITION                 NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)
           1477203      a100 benchmark_all_varian iwi5389h  RUNNING    1:27:06  20:00:00      1 tg094


⏳ Job is still running. Re-run this cell to check again.
   Estimated time: ~2-3 hours for all 4 models

📄 Recent log output (last 15 lines):
----------------------------------------------------------------------

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

Validation DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 3551.49it/s]

                                                                        
Epoch 51699: 100%|██████████| 1/1 [00:00<00:00, 10.82it/s, v_num=1477203, train_loss_step=0.904, train_loss_epoch=0.250]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Valida

## 5. Display Benchmark Results

Load and display the comprehensive metrics comparison.

In [ ]:
# Load and display latest results
import glob

# Find the latest timestamped results file
results_pattern = os.path.join(RESULTS_DIR, "benchmark_results_*_Heat_Nordbyen_*.csv")
result_files = sorted(glob.glob(results_pattern))

if not result_files:
    print("❌ ERROR: No benchmark results found!")
    print(f"   Searched for: {results_pattern}")
    print("\n💡 Solution:")
    print("   - Check cell above to see if job is still running")
    print("   - If job completed, check log files for errors")
else:
    # Load the most recent results file
    latest_results_file = result_files[-1]
    results_df = pd.read_csv(latest_results_file)
    
    print(f"\n📊 HEAT CONSUMPTION BENCHMARK RESULTS")
    print(f"Loaded from: {os.path.basename(latest_results_file)}")
    print("="*70)
    display(results_df)
    
    # Highlight best performer for each metric
    print("\n🏆 Best Performer by Metric:")
    for metric in ['MAE', 'RMSE', 'MAPE', 'MIW', 'CRPS']:
        if metric in results_df.columns:
            best_model = results_df.loc[results_df[metric].idxmin(), 'Model']
            best_value = results_df[metric].min()
            print(f"  {metric:8s}: {best_model:12s} ({best_value:.4f})")
    
    # For PICP, closer to 80% is better
    if 'PICP' in results_df.columns:
        target_picp = 80.0
        results_df['PICP_diff'] = (results_df['PICP'] - target_picp).abs()
        best_model = results_df.loc[results_df['PICP_diff'].idxmin(), 'Model']
        best_value = results_df.loc[results_df['PICP_diff'].idxmin(), 'PICP']
        print(f"  PICP    : {best_model:12s} ({best_value:.2f}% - closest to 80%)")

❌ ERROR: Benchmark results not found!
   Expected file: ../results/benchmark_results.csv

💡 Solution:
   - Check cell above to see if job is still running
   - If job completed, check log files for errors


## 6. Generate Visualizations

Create comprehensive visualizations comparing all models.

In [ ]:
# Generate visualizations using dataset-specific results folder
print("Generating visualizations...")

# Call visualize_benchmark.py with the dataset-specific results folder
result = subprocess.run(
    ["python3", "../../visualize_benchmark.py", os.path.abspath(RESULTS_DIR)],
    capture_output=True,
    text=True
)

print(result.stdout)
if result.returncode == 0:
    print("✓ Visualizations generated successfully!")
else:
    print(f"✗ Error generating visualizations:\n{result.stderr}")

### 6.1 Metrics Comparison Bar Plots

In [ ]:
metrics_plot = os.path.join(RESULTS_DIR, "benchmark_metrics_barplots.png")
if os.path.exists(metrics_plot):
    display(Image(filename=metrics_plot))
else:
    print(f"Plot not found: {metrics_plot}")

### 6.2 Box Plots - Prediction Distributions

In [ ]:
boxplot_img = os.path.join(RESULTS_DIR, "benchmark_boxplots.png")
if os.path.exists(boxplot_img):
    display(Image(filename=boxplot_img))
else:
    print(f"Plot not found: {boxplot_img}")

### 6.3 Side-by-Side Time Series Comparison

In [ ]:
sidebyside_img = os.path.join(RESULTS_DIR, "benchmark_comparison_sidebyside.png")
if os.path.exists(sidebyside_img):
    display(Image(filename=sidebyside_img))
else:
    print(f"Plot not found: {sidebyside_img}")

## 7. View Benchmark History

Track improvements over multiple runs.

In [ ]:
# View benchmark history across all runs for HEAT dataset
history_file = "../../results/benchmark_history.csv"

if os.path.exists(history_file):
    history = pd.read_csv(history_file)
    
    # Filter for heat dataset only
    heat_history = history[history['dataset'] == 'Heat (Nordbyen)']
    
    if len(heat_history) == 0:
        print("❌ No heat benchmark history found yet.")
        print("   Run a benchmark first to start tracking results.")
    else:
        print("📊 HEAT BENCHMARK HISTORY - ALL RUNS")
        print("="*70)
        print(f"Total runs recorded: {len(heat_history)}")
        print(f"Models tracked: {', '.join(heat_history['Model'].unique())}")
        print("\n" + "="*70)
        
        # Show all runs with key metrics
        display_cols = ['run_date', 'Model', 'n_epochs', 'has_hpo', 'MAE', 'RMSE', 'MAPE', 'CRPS']
        available_cols = [col for col in display_cols if col in heat_history.columns]
        display(heat_history[available_cols].sort_values('run_date', ascending=False))
        
        # Show best results ever
        print("\n🏆 BEST RESULTS EVER ACHIEVED")
        print("="*70)
        for model in heat_history['Model'].unique():
            model_history = heat_history[model_history['Model'] == model]
            print(f"\n{model}:")
            for metric in ['MAE', 'RMSE', 'MAPE', 'CRPS']:
                if metric in model_history.columns:
                    best_idx = model_history[metric].idxmin()
                    best_val = model_history.loc[best_idx, metric]
                    best_date = model_history.loc[best_idx, 'run_date']
                    print(f"  {metric:8s}: {best_val:.4f} (achieved on {best_date})")
else:
    print("❌ No benchmark history found.")
    print(f"   Expected file: {history_file}")
    print("\n💡 Run a benchmark first to start tracking results.")

## 8. Summary and Conclusions

### Key Findings:

Compare these results with the water benchmarking results to analyze domain differences.

**Next Steps:**
- Review individual model plots for detailed error analysis
- Compare with water benchmark results (in `water_centrum_benchmark/results/`)
- Consider hyperparameter optimization if results need improvement (see `hpo/` folder)
- Examine specific time periods where models differ significantly